In [9]:
from langchain_google_genai import GoogleGenerativeAI
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
import os
from dotenv import load_dotenv
from langchain import OpenAI
import openai
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Milvus
from langchain_core.documents import Document
from langchain.chains import RetrievalQA 
from langchain.memory import ConversationBufferWindowMemory
from langchain import PromptTemplate
from langchain.memory import PostgresChatMessageHistory

load_dotenv()


d:\anaconda\installed\envs\chatbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

## LLM

### Google Gemini

In [3]:

# generative model
llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=os.getenv("GOOGLE_API_KEY") , temperature=0.6)

### OpenAI LLM

In [30]:
openai.api_key = os.getenv('OPENAI_API_KEY')
llm=OpenAI(temperature=0.5)
print(llm.model_name)

## Embeddings

In [4]:
# embeddings
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=os.getenv("HUGGINGFACE_API_KEY"), model_name="sentence-transformers/all-MiniLM-l6-v2"
)

## Data Processing

In [101]:
# loader = UnstructuredURLLoader(urls=urls)
data = """
****** 
Contact details of Human Customer service representative from AudioPro Company:
 email : contact@AudioPro.in
 mobile number : 3636565642

******
Details of Product : SonicWave 2000
   - Over-ear wireless headphones with active noise cancellation.
   - Crystal-clear audio quality with deep bass and crisp highs.
   - Up to 30 hours of battery life on a single charge.
   - Adjustable headband and memory foam ear cushions for comfort.
   - Built-in microphone for hands-free calls.
   - price 11151 rupees
   - discount 10% 

******
   
   
Details of Product : SonicWave 2000
   - Over-ear wireless headphones with active noise cancellation.
   - Crystal-clear audio quality with deep bass and crisp highs.
   - Up to 30 hours of battery life on a single charge.
   - Adjustable headband and memory foam ear cushions for comfort.
   - Built-in microphone for hands-free calls.
   - price 11151 rupees
   - discount 10%

******
 Details of Product: BassBlast Pro
   - On-ear wired headphones designed for bass enthusiasts.
   - Powerful bass-driven sound with enhanced low frequencies.
   - Foldable design for easy portability and storage.
   - Adjustable padded headband for a comfortable fit.
   - Tangle-free flat cable with in-line remote and microphone.
   - price 1500 rupees
   - discount 15%
   - festival offer
******
 Details of Product: AeroFit 360
   - Sports in-ear Bluetooth headphones for active lifestyles.
   - Sweat-resistant and ergonomic design for a secure fit.
   - Immersive sound with emphasis on mids and highs.
   - Up to 10 hours of playback time with quick charge feature.
   - Ear hooks and multiple ear tip sizes included.
   - price 12000 rupees
   - discount 5%

******
 Details of Product: StudioElite 500
   - Premium studio monitor headphones for professional audio work.
   - Accurate and balanced sound reproduction for critical listening.
   - Over-ear design with plush memory foam earpads.
   - Detachable coiled cable and foldable design for convenience.
   - Gold-plated connectors for optimal signal transmission.
   - price 50005 rupees
   - discount 80%
   - festival offer

******
 Details of Product: RetroVibe 800
   - Vintage-style wireless headphones with a classic look.
   - Warm and nostalgic sound signature with modern technology.
   - Up to 25 hours of wireless playback and optional wired mode.
   - Adjustable metal sliders and faux-leather ear cushions.
   - On-ear touch controls for music playback and calls.
   - price 20151 rupees
   - discount 20%
******
 Details of Product: GamerXtreme 300
   - Gaming headphones with immersive 3D audio experience.
   - Surround sound and positional audio for competitive gaming.
   - RGB lighting effects with customizable color options.
   - Noise-canceling microphone with flip-up mute functionality.
   - Compatible with PC, consoles, and mobile devices.
   - price 25000 rupees
   - discount 30%
******
 Details of Product: KidsJamz 100
   - Child-friendly wired headphones designed for young users.
   - Volume-limited to protect young ears (85dB max volume).
   - Fun and colorful designs with adjustable headband.
   - Lightweight and durable construction for kids on the go.
   - Built-in shareport for sharing audio with a friend.
   - price 200 rupees
   - discount 15%
******
 Details of Product: TravelMate 600
   - Compact and foldable wireless headphones for travelers.
   - Active noise cancellation for a peaceful travel experience.
   - Clear voice calls with dual beamforming microphones.
   - Up to 20 hours of playback and quick charge feature.
   - Comes with a hard-shell travel case.
   - price 600 rupees
   - discount 2%
******
 Details of Product: AudiophileX 9000
   - High-fidelity open-back headphones for audiophiles.
   - Wide soundstage and exceptional detail in music playback.
   - Premium materials with genuine leather and aluminum accents.
   - Detachable oxygen-free copper cable with balanced connectors.
   - Handcrafted design for an exquisite audio experience.
   - price 90000 rupees
   - discount 70%
******
 Details of Product: ProComm 200
    - Communication headphones for professional use.
    - Clear and precise audio for conference calls and meetings.
    - Noise-canceling microphone with flexible boom arm.
    - Adjustable headband and comfortable ear cushions.
    - USB and 3.5mm connectivity options.
    - price 6000 rupees
   - discount 15%
******
 Details of Product: elite buds 5k 
    - most elite buds you have ever seen.
    - Clear and precise audio for conference calls and meetings.
    - Noise-canceling microphone with flexible boom arm.
    - Adjustable headband and comfortable ear cushions.
    - USB and 3.5mm connectivity options.
    - price 100000 rupees
   - discount 25%
"""


#### splitting data into chunks
so that each chunk can later be converted into vector.
- to achieve maximumn performance I have decided to allow overflowing chunks. 
- Using this method each chunk have only one product detail
- this is to make sure that product details are not cut in halves and spread into two vector otherwise similarity search wont work Properly

  ie. if user ask about only one product then only the vector which have the name of product would be returned. it would be really hard to infer which features are of what product if the genere is same and product details is splitted in two vectors.

In [ ]:
# split data
# text_splitter = RecursiveCharacterTextSplitter(
#     separators=['******', '\n', '.', ','
#                 ],
#     chunk_size=1000
# )


text_splitter = CharacterTextSplitter(
    separator = "******",
    chunk_size=900,
    chunk_overlap=0
)


In [102]:
dataDocument = [Document(page_content=data, metadata={"namespace": "audioPro"})]

docs = text_splitter.split_documents(dataDocument)


each chunk have only one product

In [110]:
print(docs[3].page_content)

Details of Product: BassBlast Pro
   - On-ear wired headphones designed for bass enthusiasts.
   - Powerful bass-driven sound with enhanced low frequencies.
   - Foldable design for easy portability and storage.
   - Adjustable padded headband for a comfortable fit.
   - Tangle-free flat cable with in-line remote and microphone.
   - price 1500 rupees
   - discount 15%
   - festival offer


## vector database

### creating vector database from  document

In [115]:
vector_db = Milvus.from_documents(
    docs,
    embeddings,
    collection_name="audioPro",
    connection_args={
        "uri":os.getenv("ZILLIZ_CLOUD_URI") ,
        "user":os.getenv("ZILLIZ_CLOUD_USERNAME") ,
        "password":os.getenv("ZILLIZ_CLOUD_PASSWORD") ,
        # "token": ZILLIZ_CLOUD_API_KEY,  # API key, for serverless clusters which can be used as replacements for user and password
        "secure": True,
    },
)

### loading vector database

In [9]:
vector_db = Milvus(
    embeddings,
    collection_name="audioPro",
    connection_args={
    "uri":os.getenv("ZILLIZ_CLOUD_URI") ,
    "user":os.getenv("ZILLIZ_CLOUD_USERNAME") ,
    "password":os.getenv("ZILLIZ_CLOUD_PASSWORD") ,
    # "token": ZILLIZ_CLOUD_API_KEY,  # API key, for serverless clusters which can be used as replacements for user and password
    "secure": True,
    },
)

similarity search would reaturn two or three most relevant vectors

In [10]:
query = "what is discount on elite buds"
docs = vector_db.similarity_search(query)

In [12]:
for i in range(len(docs)):
    print("\n=========================================================================================\n")
    print(docs[i].page_content)



**Model: ProComm 200**
    - Communication headphones for professional use.
    - Clear and precise audio for conference calls and meetings.
    - Noise-canceling microphone with flexible boom arm.
    - Adjustable headband and comfortable ear cushions.
    - USB and 3.5mm connectivity options.
    - price 6000 rupees
   - discount 15%

 **Model: elite buds 5k **
    - most elite buds you have ever seen.
    - Clear and precise audio for conference calls and meetings.
    - Noise-canceling microphone with flexible boom arm.
    - Adjustable headband and comfortable ear cushions.
    - USB and 3.5mm connectivity options.
    - price 100000 rupees
   - discount 25%


**Model: BassBlast Pro**
   - On-ear wired headphones designed for bass enthusiasts.
   - Powerful bass-driven sound with enhanced low frequencies.
   - Foldable design for easy portability and storage.
   - Adjustable padded headband for a comfortable fit.
   - Tangle-free flat cable with in-line remote and microphone.
  

## Chain

### V1 : creating language chain for quastion and answer

In [31]:
chain = RetrievalQA.from_llm(llm=llm, retriever=vector_db.as_retriever())
query = "what is the price of elite buds after discount"
result = chain({"query": query})
print(result)

### V2 : adding memory to chain

In [35]:


memory = ConversationBufferWindowMemory(k=3 , memory_key="history" , input_key="question")
# memory = ConversationBufferMemory(memory_key="history",input_key="question")
template = """
Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the question:
------
<ctx>
{context}
</ctx>
------
<hs>
{history}
</hs>
------
{question}
Answer:
"""
prompt = PromptTemplate(
    input_variables=["history", "context", "question"],
    template=template,
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=vector_db.as_retriever(),
    verbose=True,
    chain_type_kwargs={
        "verbose": True,
        "prompt": prompt,
        "memory": memory
    }
)

[]


### V3 : Memory with postgres

creating postgres history and integrating it with memory

In [82]:
history = PostgresChatMessageHistory(
    connection_string=os.getenv("POSTGRES_CONNECTION_STRING"),
    session_id="audioPro_user_test",
    
)


memory_postgres = ConversationBufferWindowMemory(k=3 , memory_key="history" , chat_memory= history, input_key="question")



In [83]:
template = """
Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the question:
------
<ctx>
{context}
</ctx>
------
<hs>
{history}
</hs>
------
{question}
Answer:
"""
prompt = PromptTemplate(
    input_variables=["history", "context", "question"],
    template=template,
)

posgres_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=vector_db.as_retriever(),
    verbose=True,
    chain_type_kwargs={
        "verbose": True,
        "prompt": prompt,
        "memory": memory_postgres
    }
)

In [87]:
# k=3 , memory_key="history"
res = posgres_qa("what are you selling ?")
print(res)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the question:
------
<ctx>
**Model: ProComm 200**
    - Communication headphones for professional use.
    - Clear and precise audio for conference calls and meetings.
    - Noise-canceling microphone with flexible boom arm.
    - Adjustable headband and comfortable ear cushions.
    - USB and 3.5mm connectivity options.
    - price 6000 rupees
   - discount 15%

 **Model: elite buds 5k **
    - most elite buds you have ever seen.
    - Clear and precise audio for conference calls and meetings.
    - Noise-canceling microphone with flexible boom arm.
    - Adjustable headband and comfortable ear cushions.
    - USB and 3.5mm connectivity options.
    - price 100000 rupees
   - discount 25%

**Model: StudioElite 500**
   - Premium studio

In [88]:
res = posgres_qa("what are the features of ProComm 200")
print(res["result"])



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the question:
------
<ctx>
**Model: ProComm 200**
    - Communication headphones for professional use.
    - Clear and precise audio for conference calls and meetings.
    - Noise-canceling microphone with flexible boom arm.
    - Adjustable headband and comfortable ear cushions.
    - USB and 3.5mm connectivity options.
    - price 6000 rupees
   - discount 15%

 **Model: elite buds 5k **
    - most elite buds you have ever seen.
    - Clear and precise audio for conference calls and meetings.
    - Noise-canceling microphone with flexible boom arm.
    - Adjustable headband and comfortable ear cushions.
    - USB and 3.5mm connectivity options.
    - price 100000 rupees
   - discount 25%

**Model: BassBlast Pro**
   - On-ear wired hea

In [89]:
res = posgres_qa("what is price of it ?")
print(res)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the question:
------
<ctx>
**Model: GamerXtreme 300**
   - Gaming headphones with immersive 3D audio experience.
   - Surround sound and positional audio for competitive gaming.
   - RGB lighting effects with customizable color options.
   - Noise-canceling microphone with flip-up mute functionality.
   - Compatible with PC, consoles, and mobile devices.
   - price 25000 rupees
   - discount 30%

 **Model: KidsJamz 100**
   - Child-friendly wired headphones designed for young users.
   - Volume-limited to protect young ears (85dB max volume).
   - Fun and colorful designs with adjustable headband.
   - Lightweight and durable construction for kids on the go.
   - Built-in shareport for sharing audio with a friend.
   - price 200 rupees


In [90]:
memory_postgres.buffer_as_messages

[HumanMessage(content='what are you selling ?'),
 AIMessage(content='\nWe are selling a variety of headphones, including the ProComm 200, elite buds 5k, StudioElite 500, RetroVibe 800, GamerXtreme 300, KidsJamz 100, BassBlast Pro, and AeroFit 360.'),
 HumanMessage(content='what are the features of ProComm 200'),
 AIMessage(content='The features of ProComm 200 include communication headphones for professional use, clear and precise audio for conference calls and meetings, a noise-canceling microphone with a flexible boom arm, adjustable headband and comfortable ear cushions, USB and 3.5mm connectivity options, a price of 6000 rupees, and a 15% discount.'),
 HumanMessage(content='what is price of it ?'),
 AIMessage(content='The price of the ProComm 200 headphones is 6000 rupees, with a 15% discount.')]

In [91]:
history.messages

[HumanMessage(content='how much does it cost ?'),
 AIMessage(content='The cost of the SonicWave 2000 headphones from AudioPro Company is 11,151 rupees.'),
 HumanMessage(content='what are you selling ?'),
 AIMessage(content='\nWe are selling a variety of headphones, including the ProComm 200, elite buds 5k, StudioElite 500, RetroVibe 800, GamerXtreme 300, KidsJamz 100, BassBlast Pro, and AeroFit 360.'),
 HumanMessage(content='what are the features of ProComm 200'),
 AIMessage(content='The features of ProComm 200 include communication headphones for professional use, clear and precise audio for conference calls and meetings, a noise-canceling microphone with a flexible boom arm, adjustable headband and comfortable ear cushions, USB and 3.5mm connectivity options, a price of 6000 rupees, and a 15% discount.'),
 HumanMessage(content='what is price of it ?'),
 AIMessage(content='The price of the ProComm 200 headphones is 6000 rupees, with a 15% discount.')]